In [ ]:
import tkinter as tk
from tkinter import messagebox

# ==========================================
# 1. KLASSEN UND HILFSFUNKTIONEN
# ==========================================

class Spielstein:
    """Repräsentiert ein Haus, Kirchenschiff oder Kirchturm."""
    def __init__(self, spieler_id, typ, ausrichtung='N'):
        self.spieler = spieler_id
        self.typ = typ             # 'Haus', 'Schiff', 'Turm'
        self.ausrichtung = ausrichtung

    def drehen(self):
        """Dreht den Stein nach einem Zug um 90 Grad im Uhrzeigersinn."""
        # Zyklus: N -> O -> S -> W -> N
        richtungen = ['N', 'O', 'S', 'W']
        if self.ausrichtung in richtungen:
            idx = richtungen.index(self.ausrichtung)
            self.ausrichtung = richtungen[(idx + 1) % 4]

    def get_farbe(self):
        return '#FF9999' if self.spieler == 1 else '#9999FF'

# --- KONSTANTEN ---
ROWS, COLS = 7, 7
CELL_SIZE = 90
WINDOW_SIZE = ROWS * CELL_SIZE
LINE_WIDTH = 3

OFFSET = 30

# ==========================================
# 2. POPUP FÜR RICHTUNGSWAHL
# ==========================================
class RichtungsWaehler(tk.Toplevel):
    def __init__(self, parent):
        super().__init__(parent)
        self.title("Ausrichtung wählen")
        self.geometry("250x200")
        self.ergebnis = None
        x = parent.winfo_x() + 50; y = parent.winfo_y() + 50
        self.geometry(f"+{x}+{y}")

        tk.Label(self, text="Wie soll der Stein stehen?", font=('Arial', 12)).pack(pady=20)
        btn_font = ('Arial', 10, 'bold')
        # Vertikal
        tk.Button(self, text="↑ Nord", command=lambda: self.setze('N')).pack(pady=5)
        tk.Button(self, text="↓ Süd", command=lambda: self.setze('S')).pack(pady=5)

        # Horizontal
        tk.Button(self, text="→ Ost", command=lambda: self.setze('O')).pack(pady=5)
        tk.Button(self, text="← West", command=lambda: self.setze('W')).pack(pady=5)

        self.transient(parent); self.grab_set(); self.wait_window(self)

    def setze(self, richtung):
        self.ergebnis = richtung
        self.destroy()

# ==========================================
# 3. HAUPTKLASSE SPIELBRETT
# ==========================================

class Spielbrett(tk.Canvas):

    def __init__(self, master):
        super().__init__(master, width=WINDOW_SIZE + 2 * OFFSET, height=WINDOW_SIZE + 2 * OFFSET, bg='#F0F0F0')
        self.pack()
        self.ki_aktiv = False
        self.ki_spieler_id = 2   #Spieler 2 ist KI

        self.spielfeld_matrix = [[None] * COLS for _ in range(ROWS)]
        self.unplatzierte_steine = {
            1: self._initialisiere_steine(1),
            2: self._initialisiere_steine(2)
        }
        self.pfarrer_pos = (4, 4)
        self.aktueller_spieler = 1
        self.ausgewaehlter_stein_pos = None
        self.spieler_farben = {1: 'Rot', 2: 'Blau'}

        self.draw_grid()
        self.draw_labels()
        self.initialisiere_spielfeld()
        self.bind("<Button-1>", self.handle_click)
        self.update_title()
        self.frage_spielmodus()
    def frage_spielmodus(self):
        antwort = messagebox.askquestion("Spielmodus", "Möchtest du gegen die KI spielen?")
        self.ki_aktiv = (antwort == 'yes')

    def _initialisiere_steine(self, spieler_id):
        steine = []
        steine.append(Spielstein(spieler_id, 'Turm'))
        steine.append(Spielstein(spieler_id, 'Schiff'))
        for _ in range(7): steine.append(Spielstein(spieler_id, 'Haus'))
        return steine

    def initialisiere_spielfeld(self):
        self.draw_pfarrer(self.pfarrer_pos[0], self.pfarrer_pos[1])

    # ---------------------------------------------------------
    # LOGIK-KERN: BERECHNUNG DER ERLAUBTEN ZÜGE (VEKTOR)
    # ---------------------------------------------------------

    def hat_eigene_orthogonale_nachbarn(self, row, col, spieler_id):
        nachbarn = [(row-1, col), (row+1, col), (row, col-1), (row, col+1)]
        for r, c in nachbarn:
            if 1 <= r <= ROWS and 1 <= c <= COLS:
                stein = self.spielfeld_matrix[r-1][c-1]
                if stein is not None and stein.spieler == spieler_id:
                    return True
        return False

    def get_valid_moves(self, spieler_id):
        """Berechnet alle erlaubten Züge (BIDIREKTIONALES GLEITEN)."""
        moves = []
        vorrat = self.unplatzierte_steine[spieler_id]

        # --- PHASE 1: PLATZIEREN ---
        if len(vorrat) > 0:
            naechster_stein = vorrat[0]

            # A) Kirchenteile (Ecken, Diagonal-Check)
            if naechster_stein.typ in ['Turm', 'Schiff']:
                alle_ecken = [(1, 1), (1, 7), (7, 1), (7, 7)]
                partner_map = {(1, 1): (7, 7), (7, 7): (1, 1), (1, 7): (7, 1), (7, 1): (1, 7)}

                for (r, c) in alle_ecken:
                    if self.spielfeld_matrix[r-1][c-1] is None:
                        pr, pc = partner_map[(r, c)]
                        partner_stein = self.spielfeld_matrix[pr-1][pc-1]

                        diagonale_ok = True
                        if partner_stein is not None and partner_stein.spieler != spieler_id:
                            diagonale_ok = False

                        if diagonale_ok:
                            moves.append({'type': 'place', 'pos': (r, c), 'stone_type': naechster_stein.typ})

            # B) Häuser (Frei, keine eigenen Nachbarn)
            else:
                for r in range(1, ROWS + 1):
                    for c in range(1, COLS + 1):
                        if self.spielfeld_matrix[r-1][c-1] is None and (r, c) != self.pfarrer_pos:
                            if not self.hat_eigene_orthogonale_nachbarn(r, c, spieler_id):
                                moves.append({'type': 'place', 'pos': (r, c), 'stone_type': 'Haus'})

        # --- PHASE 2: BEWEGEN (Gleiten in beide Richtungen) ---
        else:
            for r in range(1, ROWS + 1):
                for c in range(1, COLS + 1):
                    stein = self.spielfeld_matrix[r-1][c-1]

                    if stein and stein.spieler == spieler_id:

                        # Bestimme die Achsen-Richtungen basierend auf der Ausrichtung
                        achsen_richtungen = []
                        if stein.ausrichtung in ['N', 'S']:
                            # Vertikale Achse: Oben (0, -1) UND Unten (0, 1)
                            achsen_richtungen = [(0, -1), (0, 1)]
                        else:
                            # Horizontale Achse: Rechts (1, 0) UND Links (-1, 0)
                            achsen_richtungen = [(1, 0), (-1, 0)]

                        # Prüfe BEIDE Richtungen der Achse
                        for dx, dy in achsen_richtungen:
                            distanz = 1
                            while True:
                                ziel_r = r + (dy * distanz)
                                ziel_c = c + (dx * distanz)

                                # 1. Ist Ziel noch auf dem Brett?
                                if not (1 <= ziel_r <= ROWS and 1 <= ziel_c <= COLS):
                                    break # Brettende

                                ziel_inhalt = self.spielfeld_matrix[ziel_r-1][ziel_c-1]

                                # 2. Blockiert durch Pfarrer?
                                if (ziel_r, ziel_c) == self.pfarrer_pos:
                                    break

                                # 3. Blockiert durch anderen Stein?
                                if ziel_inhalt is not None:
                                    break

                                # Feld frei -> Zug hinzufügen
                                moves.append({'type': 'move', 'from': (r, c), 'to': (ziel_r, ziel_c)})

                                # Weiter gleiten
                                distanz += 1

        return moves


    
    # --------------------------
    # SIEGBEDINGUNG
    # --------------------------
    def pruefe_sieg(self, spieler_id):
        """
        Prüft, ob der Spieler die Siegbedingung erfüllt:
        - Alle eigenen Steine zusammenhängend
        - Mindestens ein Turm nicht am Rand
        - Mindestens ein Kirchenschiff vorhanden
        """
        from collections import deque

        # Spielfeldgröße
        ROWS, COLS = len(self.spielfeld_matrix), len(self.spielfeld_matrix[0])

        # Alle eigenen Steine sammeln
        steine = [(r, c) for r in range(ROWS) for c in range(COLS)
              if self.spielfeld_matrix[r][c] and self.spielfeld_matrix[r][c].spieler == spieler_id]

        if not steine:
            return False

        # ----------------------
        # 1. Zusammenhängende Gruppe prüfen (BFS)
        # ----------------------
        besuchte = [[False]*COLS for _ in range(ROWS)]
        queue = deque()
        queue.append(steine[0])
        besuchte[steine[0][0]][steine[0][1]] = True
        count = 1

        while queue:
             r, c = queue.popleft()
             for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:  # orthogonal
                nr, nc = r + dr, c + dc
                if 0 <= nr < ROWS and 0 <= nc < COLS and not besuchte[nr][nc]:
                    s = self.spielfeld_matrix[nr][nc]
                    if s and s.spieler == spieler_id:
                        queue.append((nr, nc))
                        besuchte[nr][nc] = True
                        count += 1

                 # Mindestens 9 Steine müssen zusammenhängen
        if count < 9:
            return False

        # ----------------------
        # 2. Turm und Schiff prüfen
        # ----------------------
        turm_ok = False
        schiff_ok = False
        for r, c in steine:
            s = self.spielfeld_matrix[r][c]
            if s.typ == 'Turm':
                 # Turm darf nicht am Rand stehen
                if 0 < r < ROWS-1 and 0 < c < COLS-1:
                    turm_ok = True
            elif s.typ == 'Schiff':
                schiff_ok = True

        if not turm_ok or not schiff_ok:
            return False

        # ----------------------
        # Alles erfüllt -> Sieg
        # ----------------------
        return True

    # --------------------------
    #  Bedingungen für blokierten Zug
    # --------------------------
    def ist_blockierter_stein(self, row, col):
        """Prüft, ob der Stein an dieser Position blockiert ist (kein Zug möglich)."""
        stein = self.spielfeld_matrix[row-1][col-1]
        if not stein:
            return False
        valid_moves = self.get_valid_moves(stein.spieler)
        for m in valid_moves:
            if m['type'] == 'move' and m['from'] == (row, col):
                return False
        return True
    
     
    # --------------------------
    #   Pfarer Austaush
    # --------------------------

    def tausche_pfarrer_mit(self, row, col):
        """
        Tauscht den Pfarrer mit einem blockierten Stein.
        """
        if not self.ist_blockierter_stein(row, col):
            messagebox.showwarning("Ungültig", "Dieser Stein ist nicht blockiert!")
            return False

        stein = self.spielfeld_matrix[row-1][col-1]
        if stein is None:
            return False

        pf_r, pf_c = self.pfarrer_pos

        # Lösche Visualisierung
        self.delete(f"piece_{row}_{col}")
        self.delete(f"arrow_{row}_{col}")
        self.delete(f"label_{row}_{col}")
        self.delete("pfarrer")
        self.delete("pfarrer_label")

        # Tausch in Matrix
        self.spielfeld_matrix[pf_r-1][pf_c-1] = stein
        self.spielfeld_matrix[row-1][col-1] = None

        # Update Pfarrer Position
        self.pfarrer_pos = (row, col)

        # Visualisierung
        self.draw_piece(pf_r, pf_c, stein)
        self.draw_pfarrer(row, col)


# --- KI BEWERTUNG (Inklusive Ausrichtung) ---

    def bewerte_ausrichtung(self, pos, richtung):
        """Prüft, wie viel Bewegungsfreiheit eine Ausrichtung bietet."""
        r, c = pos
        score = 0
        achsen_vektoren = [(0, -1), (0, 1)] if richtung in ['N', 'S'] else [(1, 0), (-1, 0)]
        
        for dx, dy in achsen_vektoren:
            dist = 1
            while True:
                nr, nc = r + (dy * dist), c + (dx * dist)
                if not (1 <= nr <= ROWS and 1 <= nc <= COLS) or self.spielfeld_matrix[nr-1][nc-1] or (nr, nc) == self.pfarrer_pos:
                    break
                score += 2 # Jedes freie Feld in der Achse ist gut
                dist += 1
        return score

    def bewerte_zug(self, zug, spieler_id):
        score = 0
        if zug['type'] == 'place':
            r, c = zug['pos']
            score += 10 if zug['stone_type'] in ['Turm', 'Schiff'] else 1
            score += self.naehe_zu_eigenen_steinen(r, c, spieler_id)
            score += self.sieg_nah_score(r, c)
        elif zug['type'] == 'move':
            zr, zc = zug['to']
            score += self.sieg_nah_score(zr, zc)
            score += self.synergie_mit_eigenen(zr, zc, spieler_id)
        return score

    def naehe_zu_eigenen_steinen(self, r, c, spieler_id):
        score = 0
        for dr, dc in [(0,1), (1,0), (0,-1), (-1,0)]:
            nr, nc = r + dr, c + dc
            if 1 <= nr <= ROWS and 1 <= nc <= COLS:
                s = self.spielfeld_matrix[nr-1][nc-1]
                if s and s.spieler == spieler_id: score += 1
        return score

    def sieg_nah_score(self, r, c):
        dist = abs(r - 4) + abs(c - 4)
        score = max(0, 5 - dist)
        if r in [1, ROWS] or c in [1, COLS]: score -= 6
        return score

    def synergie_mit_eigenen(self, r, c, spieler_id):
        n = self.naehe_zu_eigenen_steinen(r, c, spieler_id)
        return 6 if n >= 2 else (2 if n == 1 else -4)

    def ki_macht_zug(self):
        valid_moves = self.get_valid_moves(self.ki_spieler_id)
        if not valid_moves: return

        bester_zug = None
        beste_ausrichtung = 'N'
        max_score = -999

        for m in valid_moves:
            if m['type'] == 'place':
                for richtung in ['N', 'O', 'S', 'W']:
                    score = self.bewerte_zug(m, self.ki_spieler_id) + self.bewerte_ausrichtung(m['pos'], richtung)
                    if score > max_score:
                        max_score, bester_zug, beste_ausrichtung = score, m, richtung
            else:
                score = self.bewerte_zug(m, self.ki_spieler_id)
                # Nach Bewegung dreht sich der Stein automatisch, wir bewerten die neue Achse
                dummy_stein = Spielstein(self.ki_spieler_id, 'Haus', self.spielfeld_matrix[m['from'][0]-1][m['from'][1]-1].ausrichtung)
                dummy_stein.drehen()
                score += self.bewerte_ausrichtung(m['to'], dummy_stein.ausrichtung)
                if score > max_score:
                    max_score, bester_zug = score, m

        if bester_zug['type'] == 'place':
            r, c = bester_zug['pos']
            stein = self.unplatzierte_steine[self.ki_spieler_id].pop(0)
            stein.ausrichtung = beste_ausrichtung
            self.platziere_stein(r, c, stein)
        else:
            self.ausgewaehlter_stein_pos = bester_zug['from']
            self.fuehre_zug_aus(bester_zug['to'][0], bester_zug['to'][1])
        self.ausgewaehlter_stein_pos = None
        if self.pruefe_sieg(self.ki_spieler_id):
            messagebox.showinfo("Spiel vorbei", "Die KI hat gewonnen!")
        else:
            self.wechsle_spieler()

    # ---------------------------------------------------------
    # INTERAKTION
    # ---------------------------------------------------------

    def handle_click(self, event):
        row, col = self.get_grid_coords(event.x, event.y)
        if row is None: return

        valid_moves = self.get_valid_moves(self.aktueller_spieler)
        is_placement_phase = any(m['type'] == 'place' for m in valid_moves)
        stein_in_zelle = self.spielfeld_matrix[row-1][col-1]

            # =========================================================
            # 1️⃣ PFARRER WIRD ANGEKLICKT → PFARRER-MODUS START
             # =========================================================
        if (row, col) == self.pfarrer_pos and not is_placement_phase:
             self.ausgewaehlter_stein_pos = 'pfarrer'
             messagebox.showinfo(
                 "Pfarrer",
                 "Wähle jetzt einen blockierten Stein zum Tauschen."
             )
             return

        # --- FALL A: PLATZIEREN ---
        if is_placement_phase:
            move_match = None
            for move in valid_moves:
                if move['type'] == 'place' and move['pos'] == (row, col):
                    move_match = move
                    break

            if move_match:
                waehler = RichtungsWaehler(self.master)
                gewaehlte_richtung = waehler.ergebnis
                if gewaehlte_richtung:
                    neues_teil = self.unplatzierte_steine[self.aktueller_spieler].pop(0)
                    neues_teil.ausrichtung = gewaehlte_richtung
                    self.platziere_stein(row, col, neues_teil)
                    if self.pruefe_sieg(self.aktueller_spieler):
                        messagebox.showinfo("Spiel vorbei", f"Spieler {self.aktueller_spieler} hat gewonnen!")
                        return
                    self.wechsle_spieler()
                else: messagebox.showinfo("Info", "Abgebrochen.")
            else:
                self.zeige_platzierungs_fehler(row, col, valid_moves)

        # --- FALL B: BEWEGEN ---
        else:
            # 1.PFARRER-MODUS
            if self.ausgewaehlter_stein_pos == 'pfarrer':
                if self.ist_blockierter_stein(row, col):
                    self.tausche_pfarrer_mit(row, col)
                    self.wechsle_spieler()
                else:
                     messagebox.showwarning(
                        "Ungültig",
                        "Du musst einen blockierten Stein auswählen."
                    )

                         # zurücksetzen!
                self.ausgewaehlter_stein_pos = None
                return
            if not self.ausgewaehlter_stein_pos:
                if stein_in_zelle and stein_in_zelle.spieler == self.aktueller_spieler:
                    kann_ziehen = any(m['type'] == 'move' and m['from'] == (row, col) for m in valid_moves)
                    if kann_ziehen:
                        self.ausgewaehlter_stein_pos = (row, col)
                        self.markiere_stein(row, col, True)
                    else: messagebox.showwarning("Blockiert", "Dieser Stein kann nicht ziehen (Weg versperrt oder Brettende).")
            else:
                start_r, start_c = self.ausgewaehlter_stein_pos
                if (row, col) == (start_r, start_c):
                    self.markiere_stein(start_r, start_c, False)
                    self.ausgewaehlter_stein_pos = None
                    return

                zug_valide = any(m['type'] == 'move' and m['from'] == (start_r, start_c) and m['to'] == (row, col) for m in valid_moves)
                if zug_valide:
                    self.fuehre_zug_aus(row, col)
                    if self.pruefe_sieg(self.aktueller_spieler):
                        messagebox.showinfo("Spiel vorbei", f"Spieler {self.aktueller_spieler} hat gewonnen!")
                        return
                    self.ausgewaehlter_stein_pos = None
                    self.wechsle_spieler()
                else: messagebox.showwarning("Ungültig", "Hierhin kann der Stein nicht ziehen.")

    def zeige_platzierungs_fehler(self, r, c, valid_moves):
        if len(self.unplatzierte_steine[self.aktueller_spieler]) > 0:
            typ = self.unplatzierte_steine[self.aktueller_spieler][0].typ
            if typ in ['Turm', 'Schiff']:
                messagebox.showwarning("Regel", "Kirchenteile müssen auf eine freie Ecke!")
            elif self.spielfeld_matrix[r-1][c-1] is not None: pass
            elif self.hat_eigene_orthogonale_nachbarn(r, c, self.aktueller_spieler):
                messagebox.showwarning("Regel", "Nicht direkt neben eigene Steine bauen!")
            else: messagebox.showinfo("Info", "Ungültiges Feld.")

    def fuehre_zug_aus(self, ziel_row, ziel_col):
        start_row, start_col = self.ausgewaehlter_stein_pos
        stein = self.spielfeld_matrix[start_row-1][start_col-1]

        self.delete(f"piece_{start_row}_{start_col}")
        self.delete(f"arrow_{start_row}_{start_col}")
        self.delete(f"label_{start_row}_{start_col}")

        self.spielfeld_matrix[start_row-1][start_col-1] = None
        self.spielfeld_matrix[ziel_row-1][ziel_col-1] = stein

        stein.drehen()
        self.draw_piece(ziel_row, ziel_col, stein)

    def platziere_stein(self, row, col, stein_obj):
        self.spielfeld_matrix[row-1][col-1] = stein_obj
        self.draw_piece(row, col, stein_obj)

    def wechsle_spieler(self):
        self.aktueller_spieler = 1 if self.aktueller_spieler == 2 else 2
        self.update_title()
        # KI ist dran?
        if self.ki_aktiv and self.aktueller_spieler == self.ki_spieler_id:
            self.master.after(500, self.ki_macht_zug)
    def update_title(self):
        vorrat = self.unplatzierte_steine[self.aktueller_spieler]
        if len(vorrat) > 0: phase = f"Platzieren: {vorrat[0].typ}"
        else: phase = "Bewegen"
        self.master.title(f"Lass die Kirche im Dorf | Am Zug: {self.spieler_farben[self.aktueller_spieler]} | Phase: {phase}")

    # --- VISUALISIERUNG ---
    def draw_grid(self):
        for i in range(ROWS + 1):
            p = i * CELL_SIZE + OFFSET
            self.create_line(OFFSET, p, WINDOW_SIZE + OFFSET, p, width=LINE_WIDTH, fill="#404040")
            self.create_line(p, OFFSET, p, WINDOW_SIZE + OFFSET, width=LINE_WIDTH, fill="#404040")

    def draw_labels(self):
        for i in range(7):
            c = (i + 0.5) * CELL_SIZE + OFFSET
            self.create_text(c, OFFSET/2, text=str(i+1), fill='#333', font=('Arial', 11, 'bold'))
            self.create_text(OFFSET/2, c, text=str(i+1), fill='#333', font=('Arial', 11, 'bold'))

    def get_canvas_coords(self, row, col):
        return (col - 0.5) * CELL_SIZE + OFFSET, (row - 0.5) * CELL_SIZE + OFFSET

    def get_grid_coords(self, x, y):
        if x < OFFSET or y < OFFSET: return None, None
        c, r = int((x-OFFSET)//CELL_SIZE)+1, int((y-OFFSET)//CELL_SIZE)+1
        if 1<=r<=ROWS and 1<=c<=COLS: return r, c
        return None, None

    def draw_pfarrer(self, r, c):
        x, y = self.get_canvas_coords(r, c)
        rad = CELL_SIZE * 0.2
        self.create_oval(x-rad, y-rad, x+rad, y+rad, fill='#333', outline='black', width=2, tags="pfarrer")
        self.create_text(x, y, text="P", fill='white', font=('Arial', 14, 'bold'), tags="pfarrer_label")

    def draw_piece(self, r, c, stein):
        x_c, y_c = self.get_canvas_coords(r, c)
        farbe = stein.get_farbe()
        rad = CELL_SIZE * 0.38

        if stein.typ == 'Haus':
            self.create_rectangle(x_c - rad, y_c - rad, x_c + rad, y_c + rad, fill=farbe, outline='black', width=2, tags=f"piece_{r}_{c}")
        elif stein.typ == 'Schiff':
            self.create_oval(x_c - rad, y_c - rad, x_c + rad, y_c + rad, fill=farbe, outline='black', width=2, tags=f"piece_{r}_{c}")
        elif stein.typ == 'Turm':
            points = [x_c, y_c - rad, x_c + rad, y_c + rad, x_c - rad, y_c + rad]
            self.create_polygon(points, fill=farbe, outline='black', width=2, tags=f"piece_{r}_{c}")

        if stein.typ != 'Pfarrer':
            # VISUALISIERUNG ENTSPRICHT JETZT DER LOGIK: BEIDE RICHTUNGEN
            if stein.ausrichtung in ['N', 'S']:
                dx, dy = 0, 1 # Vertikal
            else:
                dx, dy = 1, 0 # Horizontal

            arrow_len = CELL_SIZE * 0.35
            self.create_line(x_c - dx*arrow_len, y_c - dy*arrow_len, x_c + dx*arrow_len, y_c + dy*arrow_len,
                             arrow=tk.BOTH, width=4, arrowshape=(10, 12, 5), fill='black', tags=f"arrow_{r}_{c}")

            
    def markiere_stein(self, r, c, active):
        if self.spielfeld_matrix[r-1][c-1]:
            outline = '#FFFF00' if active else 'black'
            width = 5 if active else 2
            self.itemconfig(f"piece_{r}_{c}", outline=outline, width=width)

if __name__ == '__main__':
    root = tk.Tk()
    root.title("Lass die Kirche im Dorf")
    root.geometry(f"{WINDOW_SIZE + 2*OFFSET + 20}x{WINDOW_SIZE + 2*OFFSET + 20}")
    sb = Spielbrett(root)
    root.mainloop()